In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import shutil
import os

!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

!unzip -q '/content/drive/MyDrive/MEDICAL/Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification.zip' -d '/content/d/'
!unzip -q '/content/d/ChestXRay2017.zip' -d '/content/d/dd/'

df = pd.read_csv('./covid-chestxray-dataset/metadata.csv')
selected_df = df[df.finding=="Pneumonia/Viral/COVID-19"]
selected_df = selected_df[(selected_df.view == "AP") | (selected_df.view == "PA")]

df = pd.read_csv('./covid-chestxray-dataset/metadata.csv')
selected_df = df[df.finding=="Pneumonia/Viral/COVID-19"]
selected_df = selected_df[(selected_df.view == "AP") | (selected_df.view == "PA")]

images = selected_df.filename.values.tolist()

os.makedirs('./COVID19-DATASET/train/covid19')
os.makedirs('./COVID19-DATASET/train/normal')

COVID_PATH = './COVID19-DATASET/train/covid19'
NORMAL_PATH = './COVID19-DATASET/train/normal'

for image in images:
    shutil.copy(os.path.join('./covid-chestxray-dataset/images', image), os.path.join(COVID_PATH, image))

for image in os.listdir('/content/d/dd/chest_xray/train/NORMAL')[:300]:
    shutil.copy(os.path.join('/content/d/dd/chest_xray/train/NORMAL', image), os.path.join(NORMAL_PATH, image))

TRAIN_PATH = '/content/COVID19-DATASET/train'

print("TRAIN_PATH: {}".format(TRAIN_PATH))

!unzip -q '/content/drive/MyDrive/MEDICAL/COVID-19_Radiography_Database.zip' -d '/content/covid19-radiography-database/'

os.makedirs('./COVID19-DATASET/test/covid19')
os.makedirs('./COVID19-DATASET/test/normal')

COVID_TEST = '/content/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'
NORMAL_TEST = '/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'

for image in os.listdir(COVID_TEST)[:100]:
    shutil.copy(os.path.join(COVID_TEST, image), os.path.join('./COVID19-DATASET/test/covid19', image))
for image in os.listdir(NORMAL_TEST)[:100]:
    shutil.copy(os.path.join(NORMAL_TEST, image), os.path.join('./COVID19-DATASET/test/normal', image))

TEST_PATH = '/content/COVID19-DATASET/test'


Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3641, done.
remote: Total 3641 (delta 0), reused 0 (delta 0), pack-reused 3641
Receiving objects: 100% (3641/3641), 632.96 MiB | 47.22 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.
Checking out files: 100% (1174/1174), done.
TRAIN_PATH: /content/COVID19-DATASET/train


In [ ]:
import sys
!git clone https://github.com/Michaeltaoma/SecureCovid.git
%cd SecureCovid
!git checkout dev
sys.path.append("/content/SecureCovid")

Cloning into 'SecureCovid'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 185 (delta 89), reused 128 (delta 45), pack-reused 0
Receiving objects: 100% (185/185), 64.07 KiB | 2.67 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/SecureCovid
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [ ]:
from model import pretrained
import util

import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

In [ ]:
from preprocess import preprocess
from trainer import train
from model import covid_net

trainloader, valloader, dataset_size = preprocess.load_split_train_test(TRAIN_PATH, .2, preprocess.covid_data_transforms)
class_names = trainloader.dataset.classes
dataloaders = {"train": trainloader, "val": valloader}
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
out_x = []
out_y_label = []
out_y_pred = []
out = []

In [ ]:
print("Len x: {}".format(len(out_x)))

Len x: 2526


In [ ]:
saved_path = "/content/drive/MyDrive/MEDICAL/trained/train_sunday_morningQAQ_1647194379.8894262.pth"
shadow = covid_net.CovidNet(model='small', n_classes=2)
shadow.load_state_dict(torch.load(saved_path))
shadow = shadow.to(device)

In [ ]:
def load_all_train(datadir, valid_size=0.0):
    train_data = datasets.ImageFolder(datadir,
                                      transform=preprocess.covid_data_transforms['train'])
    test_data = datasets.ImageFolder(datadir,
                                     transform=preprocess.covid_data_transforms['val'])
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    dataset_size = {"train": len(train_idx), "val": len(test_idx)}
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                                              sampler=train_sampler, batch_size=1)
    return trainloader

In [ ]:
from pathlib import Path
attack_train_set_path = Path("/content/drive/MyDrive/MEDICAL/attack_train")

test_images = datasets.ImageFolder(TEST_PATH, transform=preprocess.covid_test_transforms)
testloader = torch.utils.data.DataLoader(test_images, batch_size = 1)

all_train = load_all_train(TRAIN_PATH)

In [ ]:
with torch.no_grad():
    """
    Out-of-training-data data
    """
    for x_batch, y_batch in tqdm.tqdm(testloader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = shadow(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)

        out_x.append(x_batch.cpu().numpy())
        out_y_pred.append(y_test_pred.cpu().numpy())
        out_y_label.append(y_batch.cpu().numpy())
        out.append(0)

with torch.no_grad():
    """
    in-training-data data
    """
    for x_batch, y_batch in tqdm.tqdm(all_train, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = shadow(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)

        out_x.append(x_batch.cpu().numpy())
        out_y_pred.append(y_test_pred.cpu().numpy())
        out_y_label.append(y_batch.cpu().numpy())
        out.append(1)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/642 [00:00<?, ?it/s]

In [ ]:
util.toPickle(out_y_label, attack_train_set_path.joinpath("{}.pkl".format("y_label")))
util.toPickle(out_y_pred, attack_train_set_path.joinpath("{}.pkl".format("y_pred")))
util.toPickle(out, attack_train_set_path.joinpath("{}.pkl".format("target")))

In [ ]:
no_covid_y_pred = []
no_covid_target = []

covid_y_pred = []
covid_target = []


for label, pred, target in zip(out_y_label, out_y_pred, out):
    if label == 0:
        no_covid_y_pred.append(pred)
        no_covid_target.append(target)
    else:
        covid_y_pred.append(pred)
        covid_target.append(target)

In [ ]:
partition_path = attack_train_set_path.joinpath("partition")

util.toPickle(no_covid_y_pred, partition_path.joinpath("{}.pkl".format("no_covid_y_pred")))
util.toPickle(no_covid_target, partition_path.joinpath("{}.pkl".format("no_covid_target")))

util.toPickle(covid_y_pred, partition_path.joinpath("{}.pkl".format("covid_y_pred")))
util.toPickle(covid_target, partition_path.joinpath("{}.pkl".format("covid_target")))